In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.signal
import scipy.io
import time
from datetime import datetime
from tqdm.notebook import tqdm
from rfsoc_daq_overlay import DAQOverlay, ns_to_samp

In [ ]:
ol = DAQOverlay("hw/top_20240823_152318.bit", verbose=True, download=True) # new rx firmware, with ILA on ADC DMA intf, consolidated DMA, interrupt controller

In [ ]:
# check that pgood is fine for all of the pins. if this is nonzero, the AFE isn't working.
0x3f ^ ol.afe_power_status()

# bringup testfifo

In [ ]:
dac_min_voltage_mV = np.array([-20.2, -13.9, -20.9, -16.2, -15.1, -15.1, -19.4, -13.1])
dac_zero_voltage_mV = np.array([-2.4, 2.8, -3.0, 1.0, 1.2, 2.3, -1.1, 3.1])
dac_max_voltage_mV = np.array([15.5, 18.4, 14.2, 17.7, 17.3, 19.1, 17.6, 19.8])
dac_scale_per_mV = (0.25 - -0.25)/(dac_max_voltage_mV - dac_min_voltage_mV)
dac_bias_correction = -dac_scale_per_mV*(dac_max_voltage_mV + 2*dac_zero_voltage_mV + dac_min_voltage_mV)/8#-dac_zero_voltage_mV*dac_scale_per_mV

In [ ]:
def generate_pulse(pulse_param_ns, fsamp):
    rise_samples = ns_to_samp(pulse_param_ns[0], fsamp)
    width_samples = ns_to_samp(pulse_param_ns[1], fsamp)
    fall_samples = ns_to_samp(pulse_param_ns[2], fsamp)
    delay_samples = ns_to_samp(pulse_param_ns[3], fsamp)
    period_samples = ns_to_samp(pulse_param_ns[4], fsamp)
    data = np.zeros(period_samples)
    offset = delay_samples
    # rising edge
    data[offset:offset+rise_samples] = np.linspace(0, 2**15 - 1, rise_samples)
    offset += rise_samples
    # mesa
    data[offset:offset+width_samples] = 2**15 - 1
    offset += width_samples
    # falling edge
    data[offset:offset+fall_samples] = np.linspace(2**15 - 1, 0, fall_samples)
    return data

def get_savefile(device_name):
    return datetime.now().strftime("%Y%m%d_%H%M%S") + f'_{device_name}'

In [ ]:
def bringup_test():
    """Basic bringup test
    
        1. setup AWG to send triangle pulses on DAC0 and DAC1
        2. set 4 active channels
        3. set ADC mux to pick ADC0, diff(ADC0), ADC1, diff(ADC1) -> {0,1,2,3}
        5. bypass channel 3
        6. set thresholds to effectively disable events on ch 0,2,3
        7. set start/stop delays to be nonzero for ch 0,1
        8. set ch 0,1 to share trigger from 1, ch 2 to get trigger from itself, and ch 3 to 0 (don't care though)
        9. configure capture to trigger on DAC0 ({'0,1'b1}: OR, with mask 0x01)
        10. set VGA attenuation
        7. arm capture
        8. send start signal to AWG
        9. wait
        10. send stop to capture
        11. check write_depth
    """
    device_name = 'bringup_rx2_0'
    active_channels = 4
    adc_atten_dB = [32]*8
    ##########################################
    # SET UP AWG
    ##########################################
    print(f'setting up AWG')
    ol.stop_awg() # stop the AWG to make sure we return to DMA_IDLE state
    ol.reset_readout()
    ol.reset_capture()
    ol.set_awg_triggers([1] + [0]*7) # set the trigger to output a 1 only at the beginning of a burst, only on channel 0
    num_bursts = 1
    ol.set_awg_burst_length([num_bursts]*8) # if burst_length = 0, run for 2^64 - 1 cycles, basically forever
    # DMA buffer is allocated at num_channels x awg_frame_depth x 256b, so set frame length of awg_frame_depth
    #set_awg_frame_depth([awg_frame_depth]*num_channels)

    ### generate waveform to send ###
    # set dac scale and set mux to use AWG
    pulse_param_ns = [10, 5, 20, 0, 50]
    num_pulses = 10
    #####  SET OUTPUT AMPLITUDES ####
    bias = np.zeros(8)
    ampl = dac_scale_per_mV * 1 # 1 mV amplitude signal
    ampl[2:active_channels] = 0
    ol.set_dac_scale_offset(ampl, bias + dac_bias_correction)
    # set mux to select AWGs
    ol.dac_mux_select([0,1,2,3,4,5,6,7])
    # generate waveform
    awg_channel_frame_size = ol._awg_frame_depth_max*ol._dac_parallel_samples
    for i in range(ol._num_channels):
        pulse_period_samples = ns_to_samp(pulse_param_ns[4], ol._dac_fsamp)
        start = i*awg_channel_frame_size
        end = (i+1)*awg_channel_frame_size
        # clear buffer
        ol._awg_buffer[start:end] = 0
        for j in range(num_pulses):
            ol._awg_buffer[start:start+pulse_period_samples] = generate_pulse(pulse_param_ns, ol._dac_fsamp)
            start += pulse_period_samples
    ##########################################
    # SET UP ADC BUFFER
    ##########################################
    print(f'setting up ADC buffer')
    # set number of channels active
    ol.set_capture_channel_count(active_channels)
    # mux to select:
    #  ADC0 -> channel 0
    #  diff(ADC0) -> channel 1
    #  ADC1 -> channel 2
    #  diff(ADC1) -> channel 3
    ol.adc_mux_select([0,8,1,9]+[0]*4)
    # bypass discriminator on channel 3
    ol.bypass_discriminators(0x4)
    # set thresholds
    ol.set_discriminator_thresholds([0x80,0x0,0x80,0x80]+[0]*4,[0x80,0x0,0x80,0x80]+[0]*4)
    ol.set_discriminator_delays([5,5,0,0]+[0]*4,[5,5,0,0]+[0]*4,[0]*8)
    ol.set_discriminator_event_sources([1,1,2,0]+[0]*4)
    # set capture to trigger from AWG channel 0 only
    ol.configure_capture_trigger(0x1, 'or')
    ol.arm_capture()
    ##########################################
    # SET VGA GAIN/ATTENUATION
    ##########################################
    ol.set_vga_atten_dB(adc_atten_dB)
    ##########################################
    # START  AWG
    ##########################################
    print(f'starting AWG')
    ol.send_awg_data()
    ol.start_awg()
    time.sleep(0.01)
    print(f'DMA transfer exit code = {ol.get_dma_error()}')
    ##########################################
    # receive data
    ##########################################
    print(f'manually starting capture')
    #start_capture()
    ol.stop_capture()
    time.sleep(0.01)
    ol.start_readout()
    print(f'receiving ADC data')
    ol.receive_adc_data(False)
    return get_savefile(device_name)

In [ ]:
fname = bringup_test()